# Using FastAPI Agents

This notebook walks you through the process of setting up a basic agent with Pydantic AI and FastAPI Agents for the first time step-by-step.

To run this notebook, make sure you have `fastapi`, `fastapi-agents` and `pydantic-ai` installed (e.g. `pip install fastapi fastapi-agents pydantic-ai`).

## Create your Pydantic AI Agent

We're going to build a very simple agent to demo the combined power of these tools, a todo list manager!

Before running this, make sure `OPENAI_API_KEY` is set in your environment variables. If you need it, also set `OPENAI_BASE_URL` (e.g. if you are using models from GitHub Models).

To go to production we probably need to persist our todo lists somewhere like a database, but this is fine for a demo.

In [1]:
from typing import List
from pydantic_ai import Agent

todo_agent = Agent(
    "openai:gpt-4o-mini",
    system_prompt="You are managing the user's todo list."
)

todos: List[str] = []

@todo_agent.tool
def list_todos(context):
    return f"{len(todos)} todos: {"".join(todos) if len(todos) > 0 else "empty"}"

@todo_agent.tool
def add_todo(context, description: str):
    todos.append(description)
    return f"added {description} to todos"

@todo_agent.tool
def delete_todo(context, description: str):
    todos.remove(description)
    return f"removed {description} from todos"



## Test your agent

Let's make sure it works!

Since we are using Jupyter which uses asyncio for execution, we need to tell asyncio to nest the execution loop so we execute our Pydantic AI, which also uses asyncio. (You won't need this outside of Jupyter.)

In [2]:
import nest_asyncio
nest_asyncio.apply()

result = todo_agent.run_sync("make up 3 todos and add them then tell me what do i need to do")
result.data

"I've added the following todos for you:\n\n1. Buy groceries\n2. Complete the project report\n3. Go for a 30-minute walk\n\nYou need to complete these tasks!"

## Setup FastAPI 

Start by creating your FastAPI instance. This gives us an app we can add our routes to and ultimately run.

In [3]:
from fastapi import FastAPI
app = FastAPI()

## Setup FastAPI Agents

We need to configure our FastAPIAgents router. A router is just a place in FastAPI where routes can be defined.

Under the hood, FastAPIAgents is just a special kind of FastAPI APIRouter.

In [4]:

from fastapi_agents import FastAPIAgents

agents = FastAPIAgents(path_prefix="/agents")

Now we can register our todo agent. FastAPI Agents comes with a set of agent adapters for various frameworks. These adapters wrap the agent and convert it to FastAPI Agents standard interface.

We can then add the router to our FastAPI app. 

In [5]:
from fastapi_agents.pydantic_ai import PydanticAIAgent

agents.register("todo", PydanticAIAgent(todo_agent))
app.include_router(agents)

## Serve our agent with FastAPI Agents!

Now we can start our app, serving our API with uvicorn.

Uvicorn will find an available port and start our API.

You can reach the docs at the endpoint `/docs` to interact with your agent through the API in your browser.

In [ ]:
import uvicorn

uvicorn.run(app)

INFO:     Started server process [80740]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:65520 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:65520 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:65521 - "POST /agents/todo HTTP/1.1" 422 Unprocessable Entity
INFO:     127.0.0.1:65523 - "POST /agents/todo HTTP/1.1" 200 OK
INFO:     127.0.0.1:65525 - "POST /agents/todo HTTP/1.1" 200 OK
INFO:     127.0.0.1:65528 - "POST /agents/todo HTTP/1.1" 200 OK
